# Scrappe Lychee

Website link :https://harajukustreetwear.com/collections/all

Goal : get all the info product + each review.

+ Gets blogs information about Asian Streetwear's trend

## Import

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import urllib
import re
import requests
import csv
import time
from concurrent.futures import ThreadPoolExecutor
from selenium.webdriver.common.service import Service
from selenium.webdriver.common.service import Service
from fake_useragent import UserAgent
from urllib3.exceptions import NewConnectionError
import logging
from datetime import datetime, timedelta
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.firefox.options import Options
import pandas as pd 
import numpy as np 

import random
from selenium.webdriver.common.action_chains import ActionChains
from itertools import combinations
import time
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from fake_useragent import UserAgent

## Fonctions 

In [2]:
def initialize_driver():
    ua = UserAgent()
    user_agent = ua.random

    options = Options()
    options.add_argument('--headless')
    options.add_argument(f'user-agent={user_agent}')

    driver = webdriver.Firefox(options=options)
    return driver

In [3]:
def click_button(driver, xpath):
    try:
        button = driver.find_element(By.XPATH, xpath)
        button.click()
        return True
    except NoSuchElementException:
        print(f"Le bouton avec l'xpath '{xpath}' est introuvable.")
        return False

In [4]:
def scrape_content(driver, xpath, content_type='text', max_retries=3, timeout=8):
    for retry in range(max_retries):
        try:
            # Trouver tous les éléments correspondant à l'XPath fourni
            elements = WebDriverWait(driver, timeout).until(
                EC.presence_of_all_elements_located((By.XPATH, xpath))
            )

            # Extraire le contenu spécifié pour chaque élément
            if content_type == 'text':
                result = [element.text for element in elements]
            elif content_type == 'href':
                result = [element.get_attribute('href') for element in elements]
            else:
                raise ValueError("Type de contenu non pris en charge. Utilisez 'text' ou 'href'.")

            return result

        except TimeoutException:
            print(f"Timeout de {timeout} secondes atteint. Réessai {retry + 1}/{max_retries}.")

        except NoSuchElementException:
            print(f"Élément non trouvé. Vérifiez votre XPath. Réessai {retry + 1}/{max_retries}.")

        except StaleElementReferenceException:
            print(f"Stale Element Reference Exception. L'élément n'est plus attaché au DOM. Réessai {retry + 1}/{max_retries}.")

        except Exception as e:
            print(f"Une erreur s'est produite : {str(e)}")

    print(f"Impossible de trouver et extraire le contenu après {max_retries} tentatives.")
    return []

In [5]:
def action_aleatoire(driver):
    actions_possibles = ["scroll_full", "scroll_half", "move_to_element"]
    action_choisie = random.choice(actions_possibles)

    # Enregistrer la position actuelle de la page
    current_scroll_position = driver.execute_script("return window.pageYOffset;")

    if action_choisie == "scroll_full":
        # Action aléatoire : Faites défiler la page vers le bas
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    elif action_choisie == "scroll_half":
        # Action aléatoire : Faites défiler la moitié de la page vers le bas
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight / 2);")
    elif action_choisie == "move_to_element":
        # Action aléatoire : Bougez la souris vers un élément aléatoire (par exemple, div de class PriceInformation_classifiedPrice__b-Jae)
        wait = WebDriverWait(driver, 10)
        element_to_move = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'center')))
        driver.execute_script("arguments[0].scrollIntoView();", element_to_move)

    # Revenir à la position enregistrée
    driver.execute_script(f"window.scrollTo(0, {current_scroll_position});")

## Scrappe

In [ ]:
# Initialize the driver
driver = initialize_driver()

# Load the page
driver.get("https://lycheethelabel.com/")

In [ ]:
Plan D'action :

-Blog : cliquer sur div class header-blog := aller dans le a de la div ( le href)

sur chaque page recuperer tout les hrefs de blogs se trouvant dans : div class article__inner container  puis  h2  puis balise a  := recup des hrefs

sur chaque page de blog .   titre = by_css_selector('h1[data-mce-fragment]')
                            partie = by_css_selector('h2[data-mce-fragment]')
                            paragraphe = by_css_selector('p[data-mce-fragment]')
                            img = by_css_selector('div[data-mce-fragment]') recup limage dedans img
                            si presence ul =  by_css_selector('ul[data-mce-fragment]') 
                        sur 14 pages ( possibiltés de threadings)
                    

les urls des pages son : https://harajukustreetwear.com/collections/all?page=13 jusqua page 15

Sur page produits :   chaque balise a  class product-link ont un href := recuperer tout les hrefs et naviguer dedans

dans feuille produit :
    nom produit : h1 class title := recup text
    prix : span class money := recup text
    nombre reviews : div summary-text := recup text
    size = click sur le span de class select2-selection select2-selection--single :=  puis voir fonction en bas
    photo article : code en bas caroussel
    
    reviews
    note general = span class stars := get aria label
    
    puis pour reviews unitaire
    
    generer tout les commentaires via
    button class btn lx-btn lx-primary . click() tant quil est clickable
    
    nom profil =div class block title := text 
    date_post= div class block time
    
    image = img class action := src
    note := div class block stars:= get aria-label
    type_produit = div class value  := get text
    commentaire_post = div class pre-wrap main-text  := get text
    
    
    
            

In [ ]:
# ebauche scrip pour menu deroulant size  pareil pour color
# Trouvez l'élément du menu déroulant par son sélecteur CSS, XPath ou d'autres méthodes appropriées( class select2-selection select2-selection--single) et class pour couleur (select2-selection select2-selection--single)
dropdown_element = driver.find_element(By.ID, 'dropdown')  # Remplacez 'dropdown' par l'ID de votre menu déroulant

# Créez un objet Select pour manipuler le menu déroulant
dropdown = Select(dropdown_element)

# Récupérez toutes les options du menu déroulant
all_options = dropdown.options

# Affichez les valeurs récupérées
for option in all_options:
    print(option.text)


In [ ]:
### pour caroussel photo
# Localisez l'élément du carrousel Owl Carousel par sa classe
carousel = driver.find_element(By.CLASS_NAME, 'owl-carousel')

# Effectuez un drag down sur le carrousel pour naviguer
ActionChains(driver).drag_and_drop_by_offset(carousel, 0, 100).perform()

# Attendez que le carrousel se stabilise (ajustez le délai selon les besoins)
driver.implicitly_wait(5)

# Obtenez la liste des éléments <a> avec la classe spécifiée
thumbnails = carousel.find_elements(By.CSS_SELECTOR, 'a.thumbnail.thumbnail--media-image.selected')

# Récupérez les liens (href) des éléments <a>
for thumbnail in thumbnails:
    href = thumbnail.get_attribute('href')
    print(f"Link: {href}")
